In [1]:
import numpy as np 
import pandas as pd 
import os
import string
from string import digits
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [6]:
df=pd.read_csv("/workspaces/text-sql/experimentations/train.csv",encoding='utf-8')

In [7]:
df

,question,sql
0,Tell me what the notes are for South Australia,SELECT Notes FROM table WHERE Current slogan =...
1,What is the current series where the new serie...,SELECT Current series FROM table WHERE Notes =...
2,What is the format for South Australia?,SELECT Format FROM table WHERE State/territory...
3,Name the background colour for the Australian ...,SELECT Text/background colour FROM table WHERE...
4,how many times is the fuel propulsion is cng?,SELECT COUNT Fleet Series (Quantity) FROM tabl...
...,...,...
56350,What time was the match played with a score of...,SELECT Time FROM table WHERE Score = 3-2
56351,On which ground did the team play Aston Villa?,SELECT Ground FROM table WHERE Opponent = asto...
56352,What kind of competition was it at San Siro at...,SELECT Competition FROM table WHERE Ground = s...
56353,What is the total number of decile for the red...,SELECT COUNT Decile FROM table WHERE Name = re...


#### Checking null and dropping duplicates

In [6]:
lines.isnull().sum()

question    0
sql         0
dtype: int64

In [7]:
lines.drop_duplicates(inplace=True)

#### Converting to lower case

In [8]:
lines['question']=lines['question'].apply(lambda x: x.lower())
lines['sql']=lines['sql'].apply(lambda x: x.lower())

#### Strip Extra Spaces

In [9]:
lines['question']=lines['question'].apply(lambda x: x.strip())
lines['sql']=lines['sql'].apply(lambda x: x.strip())

#### Removing punctuations

In [10]:
exclude = set(string.punctuation)
lines['question']=lines['question'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['sql']=lines['sql'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

#### Removing + signs

In [11]:
lines['question']=lines['question'].apply(lambda x: re.sub(" +", " ", x))
lines['sql']=lines['sql'].apply(lambda x: re.sub(" +", " ", x))

#### Adding start and end to the queries

In [12]:
lines['sql'] = lines['sql'].apply(lambda x : 'START_ '+ x + ' _END')

In [13]:
all_eng_words=set()
for eng in lines['question']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [14]:
all_sql_words=set()
for i in lines['sql']:
    for word in i.split():
        if word not in all_sql_words:
            all_sql_words.add(word)

In [15]:
lines['length_question_sentence']=lines['question'].apply(lambda x:len(x.split(" ")))
lines['length_sql_sentence']=lines['sql'].apply(lambda x:len(x.split(" ")))

In [16]:
lines.head()

,question,sql,length_question_sentence,length_sql_sentence
0,tell me what the notes are for south australia,START_ select notes from table where current s...,9,11
1,what is the current series where the new serie...,START_ select current series from table where ...,13,15
2,what is the format for south australia,START_ select format from table where stateter...,7,10
3,name the background colour for the australian ...,START_ select textbackground colour from table...,9,12
4,how many times is the fuel propulsion is cng,START_ select count fleet series quantity from...,9,13


In [17]:
max_length_src=max(lines['length_sql_sentence'])
max_length_tar=max(lines['length_question_sentence'])

In [18]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_sql_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_sql_words)
num_encoder_tokens, num_decoder_tokens

(37411, 35441)

In [19]:
num_decoder_tokens += 1

In [20]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [21]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [22]:
lines = lines.sample(frac=1)

lines.head()

,question,sql,length_question_sentence,length_sql_sentence
35242,in what round was a player from michigan selected,START_ select max round from table where schoo...,9,11
20296,which species specific has a link of serversou...,START_ select species specific from table wher...,8,10
18553,what is the date of the tournament with a scor...,START_ select date from table where score 199 ...,13,10
34685,what is the amount of fa cups goals that were ...,START_ select fa cup goals from table where to...,31,20
19105,who was the democratic incumbent in the south ...,START_ select incumbent from table where party...,11,14


In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2024-01-24 05:17:11.942604: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 05:17:11.942710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 05:17:12.288375: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 05:17:13.033405: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 05:17:17.936166: W tensorflow/compiler/tf2

In [24]:
df= lines[:10000]

#### Tokenizer

In [25]:
question_tokenizer = Tokenizer()
question_tokenizer.fit_on_texts(df['question'])
question_seq = question_tokenizer.texts_to_sequences(df['question'])

sql_tokenizer = Tokenizer(filters='')
sql_tokenizer.fit_on_texts(df['sql'])
sql_seq = sql_tokenizer.texts_to_sequences(df['sql'])

In [26]:
# Pad sequences to a fixed length
max_question_length = max(len(seq) for seq in question_seq)
max_sql_length = max(len(seq) for seq in sql_seq)

question_seq_padded = pad_sequences(question_seq, maxlen=max_question_length, padding='post')
sql_seq_padded = pad_sequences(sql_seq, maxlen=max_sql_length, padding='post')

In [27]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(question_seq_padded, sql_seq_padded, test_size=0.2, random_state=42)

# Define the Seq2Seq model
latent_dim = 256

In [28]:
# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=len(question_tokenizer.word_index) + 1, output_dim=latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]


In [29]:

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=len(sql_tokenizer.word_index) + 1, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(sql_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [30]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
# Train the model
model.fit([X_train, y_train[:, :-1]], y_train[:, 1:], epochs=10, batch_size=32, validation_data=([X_val, y_val[:, :-1]], y_val[:, 1:]))

Epoch 1/10


2024-01-24 05:20:13.921250: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76671360 exceeds 10% of free system memory.
2024-01-24 05:20:14.054175: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76671360 exceeds 10% of free system memory.
2024-01-24 05:20:14.054278: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76671360 exceeds 10% of free system memory.


  1/250 [..............................] - ETA: 15:23 - loss: 9.3724 - accuracy: 0.0000e+00

2024-01-24 05:20:15.139902: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76671360 exceeds 10% of free system memory.
2024-01-24 05:20:15.308804: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76671360 exceeds 10% of free system memory.


250/250 [==============================] - 269s 1s/step - loss: 1.6186 - accuracy: 0.8313 - val_loss: 0.9798 - val_accuracy: 0.8788
Epoch 2/10
250/250 [==============================] - 263s 1s/step - loss: 0.9202 - accuracy: 0.8794 - val_loss: 0.8962 - val_accuracy: 0.8833
Epoch 3/10
250/250 [==============================] - 264s 1s/step - loss: 0.8552 - accuracy: 0.8820 - val_loss: 0.8721 - val_accuracy: 0.8844
Epoch 4/10
250/250 [==============================] - 261s 1s/step - loss: 0.8185 - accuracy: 0.8842 - val_loss: 0.8632 - val_accuracy: 0.8850
Epoch 5/10
250/250 [==============================] - 283s 1s/step - loss: 0.7866 - accuracy: 0.8875 - val_loss: 0.8483 - val_accuracy: 0.8892
Epoch 6/10
250/250 [==============================] - 260s 1s/step - loss: 0.7541 - accuracy: 0.8906 - val_loss: 0.8364 - val_accuracy: 0.8909
Epoch 7/10
250/250 [==============================] - 265s 1s/step - loss: 0.7226 - accuracy: 0.8926 - val_loss: 0.8298 - val_accuracy: 0.8923
Epoch 8/10

In [33]:
df2= lines[10000:20000]
# Tokenize new data
question_seq_df2 = question_tokenizer.texts_to_sequences(df2['question'])
sql_seq_df2 = sql_tokenizer.texts_to_sequences(df2['sql'])

# Pad sequences
question_seq_padded_df2 = pad_sequences(question_seq_df2, maxlen=max_question_length, padding='post')
sql_seq_padded_df2 = pad_sequences(sql_seq_df2, maxlen=max_sql_length, padding='post')

# Evaluate the model on the new dataset
eval_result = model.evaluate([question_seq_padded_df2, sql_seq_padded_df2[:, :-1]], sql_seq_padded_df2[:, 1:])
print("Evaluation Loss:", eval_result[0])
print("Evaluation Accuracy:", eval_result[1])

313/313 [==============================] - 102s 326ms/step - loss: 0.6572 - accuracy: 0.9025
Evaluation Loss: 0.6572026610374451
Evaluation Accuracy: 0.9024784564971924


In [42]:
# Separate Encoder Model
encoder_model = Model(encoder_inputs, encoder_states)

# Separate Decoder Model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_input = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_input
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_input, [decoder_outputs] + decoder_states
)

In [49]:
def predict_sql(input_text):
    # Tokenize the input text
    input_seq = question_tokenizer.texts_to_sequences([input_text])
    input_seq_padded = pad_sequences(input_seq, maxlen=max_question_length, padding='post')

    # Get the initial states from the encoder
    states_value = encoder_model.predict(input_seq_padded)

    # Initialize target sequence with the 'START_' token
    target_seq = np.zeros((1, 1))

    # Find the index of 'START_' in the tokenizer word_index
    start_index = sql_tokenizer.word_index.get('START_', None)

    # If 'START_' is not in word_index, use the first index
    if start_index is None:
        start_index = 1

    target_seq[0, 0] = start_index

    stop_condition = False
    decoded_sql = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_sql = sql_tokenizer.index_word.get(sampled_token_index, None)

        if sampled_sql is None or sampled_sql == '_END' or len(decoded_sql.split()) > max_sql_length:
            stop_condition = True
        else:
            decoded_sql += sampled_sql + ' '

        # Update the target sequence for the next iteration
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    # Remove the '_END' token
    decoded_sql = decoded_sql.replace('_end', '').strip()

    return decoded_sql

In [53]:
# Example prediction
example_query = "How many tackles for the player with over 0 fumble recovries and 0 forced fumbles?"
predicted_sql = predict_sql(example_query)
print("Predicted SQL:", predicted_sql)

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 14ms/step
Predicted SQL: select count year from table where team 1 racing and player billy


In [ ]:
SELECT COUNT Total FROM table WHERE Fumble rec > 0 AND Fumble force = 0